In [1]:
import tensorflow as tf
import numpy as np
from model import AttentionSeq2Seq

In [2]:
maximum_src_str_len = 40
maximum_tgt_str_len = 40
source_vocab_size = -1
target_vocab_size = -1

_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"

input_str = "I declare resumed the session of the European Parliament adjourned on Friday 17 December 1999, and I would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period."
target_str ="Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances."

def input_process(input_str):
    input_str = input_str[:-1] + " " + _EOS
    for i in range(maximum_src_str_len-len(input_str.split(" "))):
        input_str += " " + _PAD
    return input_str

def target_process(target_str):
    target_str = _GO + " " + target_str[:-1]
    target_str = target_str + " " + _EOS
    for i in range(maximum_tgt_str_len-len(target_str.split(" "))):
        target_str += " " + _PAD
    return target_str
    
input_str_list = [input_process(input_str)]
target_str_list = [target_process(target_str)]

from tensorflow.contrib import learn
source_vocab_processor = learn.preprocessing.VocabularyProcessor(maximum_src_str_len)
input_data = np.array(list(source_vocab_processor.fit_transform(input_str_list)))
source_vocab_size = len(source_vocab_processor.vocabulary_)
print("Source Vocabulary Size: {:d}".format(len(source_vocab_processor.vocabulary_)))

target_vocab_processor = learn.preprocessing.VocabularyProcessor(maximum_tgt_str_len)
target_data = np.array(list(target_vocab_processor.fit_transform(target_str_list)))
target_vocab_size = len(target_vocab_processor.vocabulary_)
print("Target Vocabulary Size: {:d}".format(len(target_vocab_processor.vocabulary_)))

source_vocab_dict = source_vocab_processor.vocabulary_._mapping
target_vocab_dict = target_vocab_processor.vocabulary_._mapping
source_vocab_rev_dict = source_vocab_processor.vocabulary_._reverse_mapping
target_vocab_rev_dict = target_vocab_processor.vocabulary_._reverse_mapping

Source Vocabulary Size: 36
Target Vocabulary Size: 36


In [3]:
input_data    = input_data.tolist()[0]
target_data   = target_data.tolist()[0]
label_data = [target_data[i+1] for i in range(len(target_data) - 1)] + [target_vocab_dict[_PAD]]

In [4]:
input_batch = []
target_batch = []
label_batch = []

for _ in range(100):
    input_batch.append(input_data)
    target_batch.append(target_data)
    label_batch.append(label_data)

In [5]:
def decode(bytes):
    sentence_marks = [0, target_vocab_dict["_PAD"], target_vocab_dict["_EOS"], target_vocab_dict["_GO"]]
    word = ""
    
    for b in bytes:
        #if not b[0] in sentence_marks:
        word += " " + target_vocab_rev_dict[b[0]]
    word += "."
    return word[1:]

In [6]:
sess = tf.Session()

batch_size = 100
enc_sequence_len = maximum_src_str_len
dec_sequence_len = maximum_tgt_str_len
enc_symbol_size = source_vocab_size
dec_symbol_size = target_vocab_size
embedding_size = 10
hidden_size = 10
layer_size = 1

model = AttentionSeq2Seq(sess,
                         batch_size,
                         enc_sequence_len, 
                         dec_sequence_len,
                         enc_symbol_size,
                         dec_symbol_size,
                         embedding_size,
                         hidden_size)

sess.run(tf.global_variables_initializer())
go_signal = []
for _ in range(batch_size):
    go_signal.append([target_vocab_dict[_GO]])

for step in range(500):
    loss, _ = model.train(input_batch, target_batch, label_batch)
    print("Average loss at step ", step, ": ", loss)
    pred = model.prediction(input_batch, go_signal)
    pred_sentence = decode(pred[0])
    print('Predicted sentence : ', pred_sentence)
    if pred_sentence == target_str:
        print("Matched!!")
        break

Average loss at step  0 :  3.58662
predict sentence :  Je déclare de la avez Je Parlement que vendredi que _EOS interrompue et vendredi passé décembre avait et vacances qui décembre le espérant vux vacances de que vous avez passé la _PAD été _EOS vacances _PAD _PAD _PAD _EOS _PAD.
Average loss at step  1 :  3.13799
predict sentence :  Je déclare reprise la tous reprise Parlement qui de avait été interrompue le vendredi 17 décembre été et je vous qui tous mes vux en vux que vous avez passé bonnes que vacances _EOS de _PAD _PAD _PAD _PAD _PAD.
Average loss at step  2 :  2.25508
predict sentence :  Je déclare reprise la avez vacances Parlement <UNK> qui avait été interrompue le vendredi 17 décembre avez et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  3 :  1.59315
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je 

predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  30 :  0.447989
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  31 :  0.447983
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  32 :  0.44798
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UN

Average loss at step  58 :  0.447958
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  59 :  0.447958
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  60 :  0.447958
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  61 :  0.447958
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le ven

Average loss at step  87 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  88 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  89 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  90 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le ven

Average loss at step  117 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  118 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  119 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  120 :  0.447957
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le

...

Average loss at step  378 :  0.447955
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  379 :  0.447955
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  380 :  0.447955
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  381 :  0.447955
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le

Average loss at step  407 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  408 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  409 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  410 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le

predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  437 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  438 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  439 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous

Average loss at step  466 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  467 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  468 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  469 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le

Average loss at step  495 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  496 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  497 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je vous <UNK> tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.
Average loss at step  498 :  0.447954
predict sentence :  Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le

최종 실험 결과….

원래 : Je déclare reprise la(session)(du)Parlement européen qui avait été interrompue le vendredi 17 décembre(dernier)et je vous (renouvelle)tous mes vux en espérant que vous avez passé  (de)  bonnes vacances.
예측 : Je déclare reprise la <UNK> <UNK> Parlement européen qui avait été interrompue le vendredi 17 décembre <UNK> et je  vous     <UNK>   tous mes vux en espérant que vous avez passé <UNK> bonnes vacances _EOS _PAD _PAD _PAD _PAD _PAD _PAD.